# ML Classification
>Learn a model that can clssify which type of given pokemon belongs to based on its **Attack** and **Defense** attributes.
<br> **Generative Model** is used in this project.
<br>This is a **binary classifier** which output is either grass type or normal type.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

In [2]:
data = pd.read_csv('/Users/yuwenchen/Pokemon-Classifier/Pokemon.csv')
data = pd.DataFrame(data)
data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [3]:
df = data[['Attack', 'Defense', 'Type 1']]

# extract all grass type pokemon 
grassType = df.loc[df['Type 1'] == 'Grass']
# extract all normal type pokemon
normalType = df.loc[df['Type 1'] == 'Normal']

# only derive 50% grass type pokemon as training set(35)
grass_training = grassType.sample(frac=0.5, replace=False, random_state=1)
# only derive 50% normal type pokemon as training set(49)
normal_training = normalType.sample(frac=0.5, replace=False, random_state=1)

# the 50% rest is testing set
grass_testing = grassType[~grassType.isin(grass_training)]
grass_testing.dropna()

normal_testing = normalType[~normalType.isin(normal_training)]
normal_testing.dropna()

,Attack,Defense,Type 1
20,45.0,40.0,Normal
21,60.0,55.0,Normal
23,80.0,80.0,Normal
24,56.0,35.0,Normal
25,81.0,60.0,Normal
26,60.0,30.0,Normal
27,90.0,65.0,Normal
44,45.0,20.0,Normal
45,70.0,45.0,Normal
58,70.0,60.0,Normal


## Classification
>#### In this project, Class 1 (C1) is grass type, Class 2 (C2) is normal type.

>### Bayes' theorem
The way to do the classfication is using **Bayes' theorem**. For example, given input *x* the probability of it belonging to class 1  is P(C1|x), which is:
>
>$$
P(C_1|x)=\frac{P(C_1)P(x|C_1)}{P(C_1)P(x|C_1)+P(C_2)P(x|C_2)}
$$ 
>
>We can caculate P(x|C1) and P(x|C2) by using Multivariate Gaussian Distrubution.

> ### Multivariate Gaussian Distrubution
Assume the training set is sampled from a Multivariate Gaussian Distrubution. Hence, once we get the mean(μ) and corvariance(∑) of this Gausssian, we can use this model (Multivariate Gausssian Distrubution) to caculate the probability of given sample belonging C1 or C2. 
>Following is the formular of Multivariate Gaussian Distrubution:
>
>$$
f_{u,\Sigma}(x)=\frac{1}{(2\pi)^{\frac{D}{2}}}\frac{1}{|\Sigma|^{\frac{1}{2}}}e^{-\frac{1}{2}(x-u)^T\Sigma^{-1}(x-u)}
$$
> Since the likelihoods of each sample are differnt, we use Maximum likelihood to find a optimal likelihood for all the sample.

> ### Maximum likelihood
The mean(μ*) and corvariance(∑*) can generate the highest probability for all the sample.
>
>The way we find μ* and ∑* is using following formulars:
(Notice that you could use differential fo find μ* and ∑* but I did not)

$$
u^*,\Sigma^*=\arg \max\limits_{u,\Sigma} L(u,\Sigma) \\
u^*=\frac{1}{79}\sum\limits_{n=1}^{79}x^n \ \ \ \ \Sigma^*=\frac{1}{79}\sum\limits_{n=1}^{79}(x^n-u^*)(x^n-u^*)^T
$$ 

> ### Therefore, P(x|C1) is:
>$$
f_{u^*,\Sigma^*}(x)=\frac{1}{(2\pi)^{\frac{D}{2}}}\frac{1}{|\Sigma^*|^{\frac{1}{2}}}e^{-\frac{1}{2}(x-u^*)^T\Sigma^{-1}(x-u^*)}
$$

In [4]:
num_g = grass_training.shape[0]
num_n = normal_training.shape[0]

# transform the Attack and Defense attributes into 2 dimensional list
grass_list = grass_training.values.T.tolist()
normal_list = normal_training.values.T.tolist()

#### Caculate P(C1) and P(C2) for Bayes' theorem

In [5]:
# 35 grass type pokemon, 49 noaml type pokemon
# grass as Class 1, noaml as Class 2
p_c1 = num_g/(num_g + num_n)
p_c2 = num_n/(num_g + num_n)

#### Caculate P(x|C1) and P(x|C2)

In [6]:
# Function that caculates Maximum Likelihood.
def getMeanCor(pokeList:list, length:int):
    
    # caculate the μ*
    u = 0
    for i in range (length):
        x_n = np.array([[pokeList[0][i]],[pokeList[1][i]]])
        u = u + x_n

    u_max = u/num_g
    u_max = u_max.astype(int)

    # caculate the ∑*
    sig = 0
    for i in range (length):
        # current
        x_n = np.array([[pokeList[0][i]],[pokeList[1][i]]])
        left = x_n - u_max
        right = left.T
        sig = sig + left * right

    sig_max = sig/num_g
    sig_max = sig_max.astype(int)
    
    u_max_flat = list(np.concatenate(u_max).flat)    
    
    return u_max_flat, sig_max

In [7]:
# get Gaussian Distrubution
# grass as Class 1
meanC1, sigC1 = getMeanCor(grass_list, num_g)
# noaml as Class 2
meanC2, sigC2 = getMeanCor(normal_list, num_n)

In [8]:
p_x_C1 = multivariate_normal(mean=meanC1, cov=sigC1)
p_x_C2 = multivariate_normal(mean=meanC2, cov=sigC2)
# print(distC1.pdf([109,88]))

## Testing Phase
> Having all the parameters for Bayes' theorem now, it will give us the probabilty of being which class once we give it a input.
>
>If the probability is greater than 0.6 then it belong to certain category.

In [9]:
def proba(w,x,y,z):
    return w*x / (w*x + y*z)

In [13]:
# grass type

test = [49,49]
output =  p_x_C1.pdf(test)*p_c1 / (p_x_C1.pdf(test)*p_c1 + p_x_C2.pdf(test)*p_c2)

In [14]:
output

0.6779507808410883